## Demonstration of the User-Based Collaborative Recommender

This system leverages collaborative filtering by analyzing user interactions, such as scroll length and read time, to identify users with similar behavior. 
Therefore, it focuses on the user-item relation.

It recommends articles that these similar users have engaged with, aiming to provide personalized suggestions. The model's performance is evaluated using MAP@K and NDCG@K metrics.



In [2]:
import sys
import os

parent_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(parent_dir)

import polars as pl
import numpy as np

from parquet_data_reader import ParquetDataReader
from models.user_based_CF import CollaborativeRecommender

pl.Config.set_tbl_cols(-1)

polars.config.Config

## Data Import and EDA

In [3]:
data_reader = ParquetDataReader()
articles_df = data_reader.read_data('../../data/articles.parquet')
train_behaviors_df = data_reader.read_data('../../data/train/behaviors.parquet')
train_history_df = data_reader.read_data('../../data/train/history.parquet')
document_vectors_df = data_reader.read_data('../../data/document_vector.parquet')

We check the size of the data. From the sizes we learn that:
<ol>
  <li>We have 20738 unique articles</li>
  <li>We have 15143 unique users</li>
  <li>We have 232887 interactions in the testset</li>
</ol> 

In [4]:
print("articles_df has the size:         ", articles_df.shape)
print("train_behaviors_df has the size:  ", train_behaviors_df.shape)
print("train_history_df has the size:    ", train_history_df.shape)
print("document_vectors_df has the size: ", document_vectors_df.shape)

articles_df has the size:          (20738, 21)
train_behaviors_df has the size:   (232887, 17)
train_history_df has the size:     (15143, 5)
document_vectors_df has the size:  (125541, 2)


### Validation Set

In [5]:
test_behaviours_df = data_reader.read_data('../../data/validation/behaviors.parquet')
test_behaviours_df.head()

impression_id,article_id,impression_time,read_time,scroll_percentage,device_type,article_ids_inview,article_ids_clicked,user_id,is_sso_user,gender,postcode,age,is_subscriber,session_id,next_read_time,next_scroll_percentage
u32,i32,datetime[μs],f32,f32,i8,list[i32],list[i32],u32,bool,i8,i8,i8,bool,u32,f32,f32
96791,null,2023-05-28 04:21:24,9.0,null,2,"[9783865, 9784591, … 9784710]",[9784696],22548,false,null,null,null,false,142,72.0,100.0
96798,null,2023-05-28 04:31:48,46.0,null,2,"[9782884, 9783865, … 9784648]",[9784281],22548,false,null,null,null,false,143,16.0,28.0
96801,null,2023-05-28 04:30:17,14.0,null,2,"[9784648, 7184889, … 9781983]",[9784444],22548,false,null,null,null,false,143,12.0,24.0
96808,null,2023-05-28 04:27:19,22.0,null,2,"[9784607, 9695098, … 9781983]",[9781983],22548,false,null,null,null,false,142,125.0,80.0
96810,null,2023-05-28 04:29:47,23.0,null,2,"[9781983, 7184889, … 9781520]",[9784642],22548,false,null,null,null,false,142,null,null


In [6]:
# Combine train and test behaviors
combined_df = pl.concat([train_behaviors_df, test_behaviours_df])

# Generate a random mask for splitting
n = combined_df.height  # Total number of rows
test_mask = np.random.rand(n) < 0.30  # 30% test, 70% train

# Apply the mask
test_behaviors_df = combined_df.filter(test_mask)
train_behaviors_df = combined_df.filter(~test_mask)

# Verify the split
print(f"Train size: {train_behaviors_df.shape[0]}, Test size: {test_behaviors_df.shape[0]}")

Train size: 333893, Test size: 143641


### Table Contents

The information on news articles. As we are going to perform user-user CF, this table is not neccesary

In [7]:
articles_df.head()

article_id,title,subtitle,last_modified_time,premium,body,published_time,image_ids,article_type,url,ner_clusters,entity_groups,topics,category,subcategory,category_str,total_inviews,total_pageviews,total_read_time,sentiment_score,sentiment_label
i32,str,str,datetime[μs],bool,str,datetime[μs],list[i64],str,str,list[str],list[str],list[str],i16,list[i16],str,i32,i32,f32,f32,str
3001353,"""Natascha var ikke den første""","""Politiet frygter nu, at Natasc…",2023-06-29 06:20:33,false,"""Sagen om den østriske Natascha…",2006-08-31 08:06:45,[3150850],"""article_default""","""https://ekstrabladet.dk/krimi/…",[],[],"[""Kriminalitet"", ""Personfarlig kriminalitet""]",140,[],"""krimi""",null,null,null,0.9955,"""Negative"""
3003065,"""Kun Star Wars tjente mere""","""Biografgængerne strømmer ind f…",2023-06-29 06:20:35,false,"""Vatikanet har opfordret til at…",2006-05-21 16:57:00,[3006712],"""article_default""","""https://ekstrabladet.dk/underh…",[],[],"[""Underholdning"", ""Film og tv"", ""Økonomi""]",414,"[433, 434]","""underholdning""",null,null,null,0.846,"""Positive"""
3012771,"""Morten Bruun fyret i Sønderjys…","""FODBOLD: Morten Bruun fyret me…",2023-06-29 06:20:39,false,"""Kemien mellem spillerne i Supe…",2006-05-01 14:28:40,[3177953],"""article_default""","""https://ekstrabladet.dk/sport/…",[],[],"[""Erhverv"", ""Kendt"", … ""Ansættelsesforhold""]",142,"[196, 199]","""sport""",null,null,null,0.8241,"""Negative"""
3023463,"""Luderne flytter på landet""","""I landets tyndest befolkede om…",2023-06-29 06:20:43,false,"""Det frække erhverv rykker på l…",2007-03-24 08:27:59,[3184029],"""article_default""","""https://ekstrabladet.dk/nyhede…",[],[],"[""Livsstil"", ""Erotik""]",118,[133],"""nyheder""",null,null,null,0.7053,"""Neutral"""
3032577,"""Cybersex: Hvornår er man utro?""","""En flirtende sms til den flott…",2023-06-29 06:20:46,false,"""De fleste af os mener, at et t…",2007-01-18 10:30:37,[3030463],"""article_default""","""https://ekstrabladet.dk/sex_og…",[],[],"[""Livsstil"", ""Partnerskab""]",565,[],"""sex_og_samliv""",null,null,null,0.9307,"""Neutral"""


Each file consists of seven days of impression logs. The train_behaviors_df table contains all information about interactions between users and items, and can be used as a basis for user-user CF. <strong>Therefore we only need this table</strong>.

In [8]:
train_behaviors_df.head()

impression_id,article_id,impression_time,read_time,scroll_percentage,device_type,article_ids_inview,article_ids_clicked,user_id,is_sso_user,gender,postcode,age,is_subscriber,session_id,next_read_time,next_scroll_percentage
u32,i32,datetime[μs],f32,f32,i8,list[i32],list[i32],u32,bool,i8,i8,i8,bool,u32,f32,f32
149474,null,2023-05-24 07:47:53,13.0,null,2,"[9778623, 9778682, … 9778728]",[9778657],139836,false,null,null,null,false,759,7.0,22.0
150528,null,2023-05-24 07:33:25,25.0,null,2,"[9778718, 9778728, … 9778682]",[9778623],143471,false,null,null,null,false,1240,287.0,100.0
153068,9778682,2023-05-24 07:09:04,78.0,100.0,1,"[9778657, 9778669, … 9778682]",[9778669],151570,false,null,null,null,false,1976,45.0,100.0
153071,9778623,2023-05-24 07:11:08,125.0,100.0,1,"[9777492, 9774568, … 9775990]",[9777492],151570,false,null,null,null,false,1976,26.0,100.0
153075,9777492,2023-05-24 07:13:58,26.0,100.0,1,"[9778500, 9776420, … 9020783]",[9777034],151570,false,null,null,null,false,1976,7.0,16.0


Each file consists of users' click histories collected over 21 days period. This table does contain the same values as the train_behaviours_df, but as that table is easier to work with we will use train_behaviours_df over this one

In [9]:
train_history_df.head()

user_id,impression_time_fixed,scroll_percentage_fixed,article_id_fixed,read_time_fixed
u32,list[datetime[μs]],list[f32],list[i32],list[f32]
13538,"[2023-04-27 10:17:43, 2023-04-27 10:18:01, … 2023-05-17 20:36:34]","[100.0, 35.0, … 100.0]","[9738663, 9738569, … 9769366]","[17.0, 12.0, … 16.0]"
14241,"[2023-04-27 09:40:18, 2023-04-27 09:40:33, … 2023-05-17 17:08:41]","[100.0, 46.0, … 100.0]","[9738557, 9738528, … 9767852]","[8.0, 9.0, … 12.0]"
20396,"[2023-04-27 12:30:44, 2023-04-27 12:31:34, … 2023-05-17 10:59:44]","[100.0, 59.0, … 13.0]","[9738760, 9738355, … 9769679]","[49.0, 34.0, … 4.0]"
34912,"[2023-04-29 07:12:49, 2023-04-29 13:01:18, … 2023-05-18 05:06:40]","[100.0, 35.0, … 27.0]","[9741802, 9741804, … 9770882]","[153.0, 7.0, … 5.0]"
37953,"[2023-04-27 19:17:10, 2023-04-27 19:17:27, … 2023-05-17 21:29:22]","[14.0, 28.0, … 18.0]","[9739205, 9739202, … 9769306]","[4.0, 16.0, … 6.0]"


List of vectors for each article. This is used to describe the items. It could be used for item-item CF, but is not relevant to user-user CF.  <strong>This table will therefore not be used</strong>

In [10]:
document_vectors_df.head()

article_id,document_vector
i32,list[f32]
3000022,"[0.065424, -0.047425, … 0.035706]"
3000063,"[0.028815, -0.000166, … 0.027167]"
3000613,"[0.037971, 0.033923, … 0.063961]"
3000700,"[0.046524, 0.002913, … 0.023423]"
3000840,"[0.014737, 0.024068, … 0.045991]"


From the analasys we see that we only need train_behaviour_df to perform user-user CF

## Preprocessing

### Remove Non-Needed Values

We see that we have several items that are not required for performing user-user CF

In [11]:
train_behaviors_df.head()

impression_id,article_id,impression_time,read_time,scroll_percentage,device_type,article_ids_inview,article_ids_clicked,user_id,is_sso_user,gender,postcode,age,is_subscriber,session_id,next_read_time,next_scroll_percentage
u32,i32,datetime[μs],f32,f32,i8,list[i32],list[i32],u32,bool,i8,i8,i8,bool,u32,f32,f32
149474,null,2023-05-24 07:47:53,13.0,null,2,"[9778623, 9778682, … 9778728]",[9778657],139836,false,null,null,null,false,759,7.0,22.0
150528,null,2023-05-24 07:33:25,25.0,null,2,"[9778718, 9778728, … 9778682]",[9778623],143471,false,null,null,null,false,1240,287.0,100.0
153068,9778682,2023-05-24 07:09:04,78.0,100.0,1,"[9778657, 9778669, … 9778682]",[9778669],151570,false,null,null,null,false,1976,45.0,100.0
153071,9778623,2023-05-24 07:11:08,125.0,100.0,1,"[9777492, 9774568, … 9775990]",[9777492],151570,false,null,null,null,false,1976,26.0,100.0
153075,9777492,2023-05-24 07:13:58,26.0,100.0,1,"[9778500, 9776420, … 9020783]",[9777034],151570,false,null,null,null,false,1976,7.0,16.0


All information that does not describe a user, or a user-item interaction can therefore be removed

In [12]:
irelevant_columns = ["impression_time", "device_type", "article_ids_inview", "article_ids_clicked", "session_id", "next_read_time", "next_scroll_percentage"]
train_behaviors_df = train_behaviors_df.drop(irelevant_columns)
train_behaviors_df.head()

impression_id,article_id,read_time,scroll_percentage,user_id,is_sso_user,gender,postcode,age,is_subscriber
u32,i32,f32,f32,u32,bool,i8,i8,i8,bool
149474,null,13.0,null,139836,false,null,null,null,false
150528,null,25.0,null,143471,false,null,null,null,false
153068,9778682,78.0,100.0,151570,false,null,null,null,false
153071,9778623,125.0,100.0,151570,false,null,null,null,false
153075,9777492,26.0,100.0,151570,false,null,null,null,false


The remaining items are the ones that can be used. But already here we see that we have several features with lacking information. We should therefore treat this

### Account for Missing Values

We see here that alot of the behaviours contain missing values. The therefore have to either remove or replace the values

In [13]:
print(train_behaviors_df.shape)
train_behaviors_df.null_count()

(333893, 10)


impression_id,article_id,read_time,scroll_percentage,user_id,is_sso_user,gender,postcode,age,is_subscriber
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,234794,0,236267,0,0,311141,327454,325188,0


In [14]:
train_behaviors_df = train_behaviors_df.filter(train_behaviors_df["article_id"].is_not_null())
print(train_behaviors_df.shape)
train_behaviors_df.null_count()

(99099, 10)


impression_id,article_id,read_time,scroll_percentage,user_id,is_sso_user,gender,postcode,age,is_subscriber
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,2519,0,0,92290,96863,95739,0


We see that of 70421, are there between 65-68000 missing values for gender, postcode and age. We therefore remove these as there is no use subsidizing them

In [15]:
train_behaviors_df = train_behaviors_df.drop(["gender", "postcode", "age"])
print(train_behaviors_df.shape)
train_behaviors_df.null_count()

(99099, 7)


impression_id,article_id,read_time,scroll_percentage,user_id,is_sso_user,is_subscriber
u32,u32,u32,u32,u32,u32,u32
0,0,0,2519,0,0,0


We still see that 2570/98967 rows are missing a scroll percentage. As this is very low (<3%) we can easily replace this. Intitially we just set scroll to 0

In [16]:
train_behaviors_df = train_behaviors_df.fill_null(strategy="zero")

### Account for Multiple Instances of the Same Article and User

By checking rows where the user_id and article_id are the same we see that we have 9855 instances where the user has read the same article multiple times

In [17]:
duplicates = train_behaviors_df.group_by(["article_id", "user_id"]).count().filter(pl.col("count") > 1)

print(duplicates)

shape: (12_250, 3)
┌────────────┬─────────┬───────┐
│ article_id ┆ user_id ┆ count │
│ ---        ┆ ---     ┆ ---   │
│ i32        ┆ u32     ┆ u32   │
╞════════════╪═════════╪═══════╡
│ 9771888    ┆ 2174314 ┆ 2     │
│ 9778845    ┆ 561475  ┆ 2     │
│ 9780769    ┆ 233128  ┆ 3     │
│ 9774079    ┆ 1482733 ┆ 2     │
│ 9776583    ┆ 1952680 ┆ 2     │
│ …          ┆ …       ┆ …     │
│ 9774363    ┆ 1379638 ┆ 2     │
│ 9772363    ┆ 1289830 ┆ 3     │
│ 9783509    ┆ 790578  ┆ 2     │
│ 9780195    ┆ 1318653 ┆ 5     │
│ 9779045    ┆ 2084203 ┆ 2     │
└────────────┴─────────┴───────┘


C:\Users\magnu\AppData\Local\Temp\ipykernel_7392\4233841252.py:1: DeprecationWarning: `GroupBy.count` is deprecated. It has been renamed to `len`.
  duplicates = train_behaviors_df.group_by(["article_id", "user_id"]).count().filter(pl.col("count") > 1)


We see that we need to combine these duplicate rows. We therefore propose that for multiple instances of the same article and user, we combine the readtime and select the largest scroll percentage. This way we can preserve the data without having duplicates

In [18]:
train_behaviors_df = (
    train_behaviors_df
    .group_by(["article_id", "user_id"])
    .agg(
        pl.col("read_time").product().alias("total_readtime"),  # Multiply all readtime values
        pl.col("scroll_percentage").max().alias("max_scroll")  # Select the largest scroll percentage
    )
)

print(train_behaviors_df)

shape: (79_516, 4)
┌────────────┬─────────┬────────────────┬────────────┐
│ article_id ┆ user_id ┆ total_readtime ┆ max_scroll │
│ ---        ┆ ---     ┆ ---            ┆ ---        │
│ i32        ┆ u32     ┆ f32            ┆ f32        │
╞════════════╪═════════╪════════════════╪════════════╡
│ 9775939    ┆ 588610  ┆ 1653.0         ┆ 100.0      │
│ 9772045    ┆ 1822997 ┆ 39.0           ┆ 100.0      │
│ 9790532    ┆ 1504982 ┆ 97.0           ┆ 100.0      │
│ 9774065    ┆ 799160  ┆ 59.0           ┆ 100.0      │
│ 9778731    ┆ 2208725 ┆ 18.0           ┆ 100.0      │
│ …          ┆ …       ┆ …              ┆ …          │
│ 9783667    ┆ 1172688 ┆ 34.0           ┆ 100.0      │
│ 9786204    ┆ 1519967 ┆ 77.0           ┆ 100.0      │
│ 9785260    ┆ 294421  ┆ 114.0          ┆ 0.0        │
│ 9776315    ┆ 200296  ┆ 14.0           ┆ 100.0      │
│ 9669548    ┆ 1229418 ┆ 67.0           ┆ 100.0      │
└────────────┴─────────┴────────────────┴────────────┘


## Model Fit

This first model uses readtime and read percentage interactions to compare the user interactions 

In [19]:
recommender = CollaborativeRecommender(train_behaviors_df)
recommender.fit()

{588610: [(2144622, np.float64(0.9998095143964887)),
  (258004, np.float64(0.9998095143964886)),
  (1124956, np.float64(0.9998095143964886)),
  (396695, np.float64(0.9998095143964886)),
  (2119221, np.float64(0.9998095143964886)),
  (453050, np.float64(0.9998095143964886)),
  (2007587, np.float64(0.9998095143964885)),
  (2121304, np.float64(0.9998095139208327)),
  (713044, np.float64(0.9998095092047129)),
  (1452576, np.float64(0.9997927890211907))],
 1822997: [(638144, np.float64(0.7090626859628042)),
  (2144548, np.float64(0.7090626859628042)),
  (2214674, np.float64(0.7090626859628041)),
  (228939, np.float64(0.7090626859628041)),
  (1772379, np.float64(0.7090626859628041)),
  (1219640, np.float64(0.7090626859628041)),
  (1781364, np.float64(0.7090626859614421)),
  (1119853, np.float64(0.7087487971241648)),
  (1874530, np.float64(0.7085543173082282)),
  (1421019, np.float64(0.6231330510353656))],
 1504982: [(1830221, np.float64(0.7162139814963396)),
  (1958444, np.float64(0.71621398

This first model just compares all artilces read by users when comparing users

In [20]:
binary_recommender = CollaborativeRecommender(train_behaviors_df, binary_model=True)
binary_recommender.fit()

{588610: [(897604, np.float64(0.35355339059327373)),
  (258004, np.float64(0.35355339059327373)),
  (1661100, np.float64(0.35355339059327373)),
  (2007587, np.float64(0.35355339059327373)),
  (396695, np.float64(0.35355339059327373)),
  (1776803, np.float64(0.35355339059327373)),
  (823907, np.float64(0.35355339059327373)),
  (508567, np.float64(0.35355339059327373)),
  (453050, np.float64(0.35355339059327373)),
  (593171, np.float64(0.35355339059327373))],
 1822997: [(870160, np.float64(0.35355339059327373)),
  (1726085, np.float64(0.35355339059327373)),
  (1192528, np.float64(0.35355339059327373)),
  (415516, np.float64(0.35355339059327373)),
  (1664278, np.float64(0.35355339059327373)),
  (801768, np.float64(0.35355339059327373)),
  (2144548, np.float64(0.35355339059327373)),
  (1792068, np.float64(0.35355339059327373)),
  (310519, np.float64(0.35355339059327373)),
  (1994147, np.float64(0.35355339059327373))],
 1504982: [(1454202, np.float64(0.2461829819586654)),
  (503743, np.floa

Of the original 15143 users, only 9194 can be accounted for with the current solution. This should be changed in the future

## Model Presentation

### Article Recommendation

In [21]:
for user in [630220, 620796, 1067393, 1726258, 17205]:
    print("reccomended for user ", user, ": ", recommender.recommend_n_articles(user_id=user, n=5, allow_read_articles=True))

reccomended for user  630220 :  [9787465, 9789418, 9772095, 9786351, 9757574]
reccomended for user  620796 :  [9783865, 9785475, 9771995, 9765753, 9774079]
reccomended for user  1067393 :  [9771113, 9774789, 9785835, 9785424, 9784275]
reccomended for user  1726258 :  [9774142, 9771224, 9789997, 9790756, 9791050]
reccomended for user  17205 :  [9780325, 9774764, 9776917, 9770886, 9781906]


In [22]:
for user in [630220, 620796, 1067393, 1726258, 17205]:
    print("reccomended for user ", user, ": ", binary_recommender.recommend_n_articles(user_id=user, n=5, allow_read_articles=True))

reccomended for user  630220 :  [9783334, 9771948, 9779860, 9782722, 9780849]
reccomended for user  620796 :  [9771686, 9774079, 9780280, 9776341, 9776223]
reccomended for user  1067393 :  [9787098, 9771113, 9778710, 9098807, 9782745]
reccomended for user  1726258 :  [9771224, 9787769, 9776223, 9786209, 9789475]
reccomended for user  17205 :  [9771846, 9779269, 9772925, 9785668, 9779674]


### Evaluation Scores

#### Without the Ability to Recommend Read Articles

The complex model only reccomending articles the user has not yet read

In [23]:
results = recommender.evaluate_recommender(test_behaviours_df, k=100, n_jobs=4, user_sample=200, allow_read_articles=False)
results

{'MAP@K': np.float64(0.0008522727272727274),
 'NDCG@K': np.float64(0.0022860400017675714)}

The binary reccomender model only reccomending articles the user has not yet read

In [24]:
results = binary_recommender.evaluate_recommender(test_behaviours_df, k=100, n_jobs=4, user_sample=200, allow_read_articles=False)
results

{'MAP@K': np.float64(0.00022988505747126436),
 'NDCG@K': np.float64(0.0005573085943802735)}

#### With the Ability to Recommend Previously Read Articles

The complex model reccomending articles the user, even if they have read them before

In [25]:
results = recommender.evaluate_recommender(test_behaviours_df, k=100, n_jobs=4, user_sample=200, allow_read_articles=True)
results

{'MAP@K': np.float64(0.011477272727272727),
 'NDCG@K': np.float64(0.24747023132549859)}

The binary reccomender model reccomending articles the user, even if they have read them before

In [26]:
results = binary_recommender.evaluate_recommender(test_behaviours_df, k=100, n_jobs=4, user_sample=200, allow_read_articles=True)
results

{'MAP@K': np.float64(0.022366863905325444),
 'NDCG@K': np.float64(0.3109340296481526)}

## Model Experimentation

In [27]:
test_user_id = 630220

predictions = recommender.recommend_n_articles(user_id=test_user_id, n=1000, allow_read_articles=True)
results = set(test_behaviours_df.filter(pl.col("user_id") == test_user_id)["article_id"])

print(results)
print(predictions)

for prediction in predictions:
    if prediction in results:
        print("Yes")

{9786243, 9787524, 9781902, 9784591, 9783824, 9786111, 9776916, 9779615, 9788705, 9789473, 9428643, 9783334, 9782315, 9756075, 9787441, 9782722, 9786821, 9782726, 9786566, 9789896, 9787465, 9788362, 9791049, 9782092, 9780815, None, 9783509, 9772508, 9786718, 9786719, 9787487, 9790942, 9783655, 9786351, 9780849, 9781875, 9788661, 9781878, 9787510, 9786618, 9673979, 9780348, 9781887}
[9787465, 9789418, 9772095, 9786351, 9757574, 9769504, 9771916, 9778628, 9781558, 9776234, 9776497, 9790822, 9786378, 9788197, 9786932, 9788894, 9788043, 9787332, 9779860, 9781086, 9784679, 9787441, 9773282, 9789423, 9786289, 9773356, 9773292, 9789694, 9786306, 9782993, 9772000, 9770288, 9786381, 9774580, 9788107, 9784879, 9781362, 9782421, 9774461, 9788524, 9771903, 9774297, 9778448, 9785350, 9782423]
Yes
Yes
Yes


In [28]:
test_user_id = 630220

predictions = recommender.recommend_n_articles(user_id=test_user_id, n=1000, allow_read_articles=True)
results = set(test_behaviours_df.filter(pl.col("user_id") == test_user_id)["article_id"])

print(results)
print(predictions)

for prediction in predictions:
    if prediction in results:
        print("Yes")

{9786243, 9787524, 9781902, 9784591, 9783824, 9786111, 9776916, 9779615, 9788705, 9789473, 9428643, 9783334, 9782315, 9756075, 9787441, 9782722, 9786821, 9782726, 9786566, 9789896, 9787465, 9788362, 9791049, 9782092, 9780815, None, 9783509, 9772508, 9786718, 9786719, 9787487, 9790942, 9783655, 9786351, 9780849, 9781875, 9788661, 9781878, 9787510, 9786618, 9673979, 9780348, 9781887}
[9787465, 9789418, 9772095, 9786351, 9757574, 9769504, 9771916, 9778628, 9781558, 9776234, 9776497, 9790822, 9786378, 9788197, 9786932, 9788894, 9788043, 9787332, 9779860, 9781086, 9784679, 9773282, 9787441, 9789423, 9786289, 9773292, 9773356, 9786306, 9789694, 9782993, 9772000, 9770288, 9786381, 9774580, 9784879, 9788107, 9781362, 9774461, 9782421, 9788524, 9771903, 9774297, 9778448, 9785350, 9782423]
Yes
Yes
Yes


In [29]:
from utils.evaluation import perform_model_evaluation

matrics = perform_model_evaluation(recommender, test_behaviours_df, k=5)
matrics

{'precision@k': np.float64(0.0010559249120062574),
 'recall@k': np.float64(0.0004410247825037336),
 'fpr@k': np.float64(0.0014870290511534074)}

### Carbon Footprint
This section creates an emissions.csv file in the "output"-folder
It utilizes the code_carbon (`codecarbon EmissionsTracker`) to record the carbon footprint of the `fit` and the `recommend` methods of the model.

In [30]:
from utils.evaluation import track_model_energy

print("\nCarbon footprint of the recommender:")
footprint = track_model_energy(recommender, "user_based", user_id=test_user_id, n=5)
footprint

[codecarbon INFO @ 17:45:28] [setup] RAM Tracking...
[codecarbon INFO @ 17:45:28] [setup] CPU Tracking...
[codecarbon WARNING @ 17:45:28] No CPU tracking mode found. Falling back on CPU constant mode. 
 Windows OS detected: Please install Intel Power Gadget to measure CPU




Carbon footprint of the recommender:


[codecarbon WARNING @ 17:45:30] We saw that you have a 13th Gen Intel(R) Core(TM) i7-13700H but we don't know it. Please contact us.
[codecarbon INFO @ 17:45:30] CPU Model on constant consumption mode: 13th Gen Intel(R) Core(TM) i7-13700H
[codecarbon INFO @ 17:45:30] [setup] GPU Tracking...
[codecarbon INFO @ 17:45:30] No GPU found.
[codecarbon INFO @ 17:45:30] >>> Tracker's metadata:
[codecarbon INFO @ 17:45:30]   Platform system: Windows-10-10.0.26100-SP0
[codecarbon INFO @ 17:45:30]   Python version: 3.11.9
[codecarbon INFO @ 17:45:30]   CodeCarbon version: 2.8.3
[codecarbon INFO @ 17:45:30]   Available RAM : 15.731 GB
[codecarbon INFO @ 17:45:30]   CPU count: 20
[codecarbon INFO @ 17:45:30]   CPU model: 13th Gen Intel(R) Core(TM) i7-13700H
[codecarbon INFO @ 17:45:30]   GPU count: None
[codecarbon INFO @ 17:45:30]   GPU model: None
[codecarbon INFO @ 17:45:33] Saving emissions data to file c:\Users\magnu\NewDesk\An.sys\TDT4215\recommender_system\demostrations\output\user_based_fit_

{'fit': ({588610: [(2144622, np.float64(0.9998095143964887)),
    (258004, np.float64(0.9998095143964886)),
    (1124956, np.float64(0.9998095143964886)),
    (396695, np.float64(0.9998095143964886)),
    (2119221, np.float64(0.9998095143964886)),
    (453050, np.float64(0.9998095143964886)),
    (2007587, np.float64(0.9998095143964885)),
    (2121304, np.float64(0.9998095139208327)),
    (713044, np.float64(0.9998095092047129)),
    (1452576, np.float64(0.9997927890211907))],
   1822997: [(638144, np.float64(0.7090626859628042)),
    (2144548, np.float64(0.7090626859628042)),
    (2214674, np.float64(0.7090626859628041)),
    (228939, np.float64(0.7090626859628041)),
    (1772379, np.float64(0.7090626859628041)),
    (1219640, np.float64(0.7090626859628041)),
    (1781364, np.float64(0.7090626859614421)),
    (1119853, np.float64(0.7087487971241648)),
    (1874530, np.float64(0.7085543173082282)),
    (1421019, np.float64(0.6231330510353656))],
   1504982: [(1830221, np.float64(0.7162